In [1]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T

import lasagne

theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
path="data/"
data=pd.read_csv(path+"input_train.csv", index_col="ID")
X_ACC=data.filter(regex= "ACC*")
X_EEG=data.filter(regex= "EEG*")
labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")

In [3]:
data_test=pd.read_csv(path+"input_test.csv", index_col="ID")
X_ACC_TEST=data_test.filter(regex= "ACC*")
X_EEG_TEST=data_test.filter(regex= "EEG*")

In [4]:
from sklearn.cross_validation import train_test_split
def train_val_test_split(X):    
    X_train, X_test = train_test_split(X, test_size=0.10, random_state=42)
    X_train, X_val = train_test_split(X_train, test_size=0.10, random_state=42)
    return X_train, X_val, X_test

In [5]:
X_ACC_train, X_ACC_val, X_ACC_test=train_val_test_split(X_ACC)
X_EEG_train, X_EEG_val, X_EEG_test=train_val_test_split(X_EEG)
y_train, y_val, y_test=train_val_test_split(labels["TARGET"])

In [6]:
X_ACC_TEST.shape, X_ACC.shape

((30458, 450), (31129, 450))

In [7]:
X_ACC_TEST.shape

(30458, 450)

In [8]:
from sklearn.preprocessing import StandardScaler

scaler_ACC_train = StandardScaler().fit(X_ACC_train)
scaler_EEG_train= StandardScaler().fit(X_EEG_train)
scaler_ACC_predict = StandardScaler().fit(X_ACC)
scaler_EEG_predict= StandardScaler().fit(X_EEG)

X_ACC_train=scaler_ACC_train.transform(X_ACC_train)
X_ACC_val=scaler_ACC_train.transform(X_ACC_val)
X_ACC_test=scaler_ACC_train.transform(X_ACC_test)

X_EEG_train=scaler_EEG_train.transform(X_EEG_train)
X_EEG_val=scaler_EEG_train.transform(X_EEG_val)
X_EEG_test=scaler_EEG_train.transform(X_EEG_test)

X_ACC=scaler_ACC_predict.transform(X_ACC)
X_EEG=scaler_EEG_predict.transform(X_EEG)

X_ACC_TEST=scaler_ACC_predict.transform(X_ACC_TEST)
X_EEG_TEST=scaler_EEG_predict.transform(X_EEG_TEST)

In [9]:
def reshape(X, n_channels):
    return X.reshape(X.shape[0],n_channels,X.shape[1]/n_channels)

In [10]:
%pdb

Automatic pdb calling has been turned ON


In [11]:
X_EEG_train=reshape(X_EEG_train,1)
X_EEG_val=reshape(X_EEG_val,1)
X_EEG_test=reshape(X_EEG_test,1)
X_EEG=reshape(X_EEG,1)
X_EEG_TEST=reshape(X_EEG_TEST,1)

X_ACC_train=reshape(X_ACC_train,3)
X_ACC_val=reshape(X_ACC_val,3)
X_ACC_test=reshape(X_ACC_test,3)
X_ACC=reshape(X_ACC,3)
X_ACC_TEST=reshape(X_ACC_TEST,3)

In [12]:
# Prepare Theano variables for inputs and targets
input_var_EEG = T.tensor3('inputs')
input_var_ACC = T.tensor3('inputs')
target_var = T.ivector('targets')
# Create neural network model

In [13]:
import time
from sklearn.base import BaseEstimator
def iterate_minibatches(input_1, input_2, targets, batchsize, shuffle=False):
    assert len(input_1) == len(targets)
    assert len(input_2) == len(targets)
    if shuffle:
        indices = np.arange(len(input_1))
        np.random.shuffle(indices)
    for start_idx in range(0, len(input_1) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield input_1[excerpt], input_2[excerpt], targets[excerpt]

"""Default variables"""
num_epochs = 15
EEG_shape_1=X_EEG_train.shape[1]
EEG_shape_2=X_EEG_train.shape[2]
ACC_shape_1=X_ACC_train.shape[1]
ACC_shape_2=X_ACC_train.shape[2]
n_filters_EEG_1=32
#n_filters_EEG_2=
n_filters_ACC_1=16
#n_filters_ACC_2=
filter_size_EEG_1=20
#filter_size_EEG_2=
filter_size_ACC_1=6
#filter_size_ACC_2=
pool_size_EEG=5
pool_size_ACC=2
drop_out=0.5
init_learning_rate=0.01
num_units_EEG=1000
num_units_ACC=100
num_units_pool=2000
num_classes=5

class CNN(BaseEstimator):
    def __init__(
        self,
        input_var_EEG, 
        input_var_ACC, 
        target_var,
        num_epochs
    ):
        #setting variables
        self.input_var_EEG=input_var_EEG
        self.input_var_ACC=input_var_ACC
        self.target_var=target_var    
        self.num_epochs=num_epochs
        
        #The network
        #Two initialisation layers
        #One for EEG
        self.l_in1 = lasagne.layers.InputLayer(
            shape=(None,EEG_shape_1, EEG_shape_2),
            input_var=self.input_var_EEG
        )
        #One for ACC
        self.l_in2 = lasagne.layers.InputLayer(
            shape=(None,ACC_shape_1, ACC_shape_2),
            input_var=self.input_var_ACC
        )
        #Two convolutional layers to treat each signal separatedly
        self.l_conv1_1 = lasagne.layers.Conv1DLayer(
            self.l_in1, 
            num_filters=n_filters_EEG_1, 
            filter_size=filter_size_EEG_1,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotNormal()
        )
        self.l_conv2_1 = lasagne.layers.Conv1DLayer(
            self.l_in2, 
            num_filters=n_filters_ACC_1, 
            filter_size=filter_size_ACC_1,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotNormal()
        )
        #And again another 2 layers
        """l_conv1_2 = lasagne.layers.Conv1DLayer(
            l_conv1_1, num_filters=16, filter_size=50,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
        l_conv2_2 = lasagne.layers.Conv1DLayer(
            l_conv2_1, num_filters=16, filter_size=5,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
        """
        #And two pooling layers
        """l_pool1 = lasagne.layers.MaxPool1DLayer(l_conv1_2, pool_size=25)
        l_pool2 = lasagne.layers.MaxPool1DLayer(l_conv2_2, pool_size=2)
        """
        self.l_pool1 = lasagne.layers.MaxPool1DLayer(
            self.l_conv1_1, 
            pool_size=pool_size_EEG
        )
        self.l_pool2 = lasagne.layers.MaxPool1DLayer(
            self.l_conv2_1, 
            pool_size=pool_size_ACC
        )    
        #Two dense layers for each signal
        self.l_dense_1_1 = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(
                self.l_pool1, 
                p=drop_out
            ),
            num_units=num_units_EEG,
            nonlinearity=lasagne.nonlinearities.rectify
        )
        self.l_dense_2_1 = lasagne.layers.DenseLayer(
                lasagne.layers.dropout(
                self.l_pool2, 
                p=drop_out
            ),
            num_units=num_units_ACC,
            nonlinearity=lasagne.nonlinearities.rectify
        )
        #use neurons from both signals to predict
        #concatenate neurons
        self.l_concat = lasagne.layers.ConcatLayer(
            [
                self.l_dense_1_1, 
                self.l_dense_2_1
            ], 
            axis=1
        )
        #dense layer with all neurons
        self.l_dense_2 = lasagne.layers.DenseLayer(
                lasagne.layers.dropout(
                self.l_concat, 
                p=drop_out
            ),
            num_units=num_units_pool,
            nonlinearity=lasagne.nonlinearities.rectify
        )
        #Output layer
        self.l_output = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(
                self.l_dense_2, 
                p=drop_out
            ),
            num_units=num_classes,
            nonlinearity=lasagne.nonlinearities.softmax
        )
        self.prediction = lasagne.layers.get_output(self.l_output)
        self.loss = lasagne.objectives.multiclass_hinge_loss(self.prediction, target_var)
        self.loss = self.loss.mean()
        self.params = lasagne.layers.get_all_params(self.l_output, trainable=True)
        self.updates = lasagne.updates.adadelta(
            self.loss, 
            self.params, 
            learning_rate=init_learning_rate
        )
        self.test_prediction = lasagne.layers.get_output(self.l_output, deterministic=True)
        self.test_loss = lasagne.objectives.multiclass_hinge_loss(self.test_prediction,
                                                                target_var)
        self.test_loss = self.test_loss.mean()
        self.test_acc = T.mean(T.eq(T.argmax(self.test_prediction, axis=1), self.target_var),
                  dtype=theano.config.floatX)
        self.train_fn =theano.function(
            [self.input_var_EEG, self.input_var_ACC, self.target_var], 
            self.loss, 
            updates=self.updates
        )
        self.val_fn = theano.function(
            [self.input_var_EEG, 
             self.input_var_ACC,
             self.target_var
            ], 
            [self.test_loss, self.test_acc])
    def fit(self,X_EEG_train,X_ACC_train,X_EEG_val,X_ACC_val,y_train,y_val):

        for epoch in range(self.num_epochs):
            # In each epoch, we do a full pass over the training data:
            train_err = 0
            train_batches = 0
            start_time = time.time()
            for batch in iterate_minibatches(X_EEG_train, X_ACC_train, y_train, 64, shuffle=True):
                input_1, input_2, targets = batch
                targets = targets.astype(np.int32)
                train_err += self.train_fn(input_1, input_2, targets)
                train_batches += 1
                if train_batches % 100 == 0:
                    print "epoch {}, train batches{}".format(epoch,train_batches) 

            # And a full pass over the validation data:
            val_err = 0
            val_acc = 0
            val_batches = 0
            for batch in iterate_minibatches(X_EEG_val, X_ACC_val, y_val, 128, shuffle=False):
                input_1, input_2, targets = batch
                targets = targets.astype(np.int32)
                err, acc = self.val_fn(input_1, input_2, targets)
                val_err += err
                val_acc += acc
                val_batches += 1

            # Then we print the results for this epoch:
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - start_time))
            print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
            print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
            print("  validation accuracy:\t\t{:.2f} %".format(
                val_acc / val_batches * 100))
        return self
        
    def predict_proba(self,X_EEG_test, X_ACC_test):
        net_output=lasagne.layers.get_output(self.l_output, deterministic=True)
        prediction_fn = theano.function([input_var_EEG, input_var_ACC], [net_output])
        prediction =  prediction_fn(X_EEG_test, X_ACC_test)
        return prediction[0]

    def predict(self,X_EEG_test, X_ACC_test):
         return np.argmax(self.predict_proba(X_EEG_test, X_ACC_test), axis=1)
        

In [14]:
cnn=CNN(input_var_EEG, 
        input_var_ACC, 
        target_var,
        num_epochs
       )

In [15]:
cnn.fit(X_EEG_train, X_ACC_train, X_EEG_val, X_ACC_val, y_train, y_val)

epoch 0, train batches100
epoch 0, train batches200
epoch 0, train batches300
Epoch 1 of 15 took 558.624s
  training loss:		0.979565
  validation loss:		0.926918
  validation accuracy:		57.66 %
epoch 1, train batches100
epoch 1, train batches200
epoch 1, train batches300
Epoch 2 of 15 took 557.725s
  training loss:		0.895965
  validation loss:		0.858384
  validation accuracy:		58.44 %
epoch 2, train batches100
epoch 2, train batches200
epoch 2, train batches300
Epoch 3 of 15 took 556.847s
  training loss:		0.848248
  validation loss:		0.835154
  validation accuracy:		58.26 %
epoch 3, train batches100
epoch 3, train batches200
epoch 3, train batches300
Epoch 4 of 15 took 556.360s
  training loss:		0.828741
  validation loss:		0.818017
  validation accuracy:		59.38 %
epoch 4, train batches100
epoch 4, train batches200
epoch 4, train batches300
Epoch 5 of 15 took 556.145s
  training loss:		0.810014
  validation loss:		0.811293
  validation accuracy:		59.60 %
epoch 5, train batches100
epoc

CNN(input_var_ACC=inputs, input_var_EEG=inputs, num_epochs=15,
  target_var=targets)

In [16]:
probas=cnn.predict_proba(X_EEG_test, X_ACC_test)

In [22]:
probas

array([[  6.77855810e-05,   4.82036697e-05,   9.99730767e-01,
          3.43746934e-05,   1.18869070e-04],
       [  9.91853397e-04,   8.18514844e-04,   9.96939142e-01,
          1.78721561e-04,   1.07176800e-03],
       [  2.36951934e-02,   1.71426373e-02,   1.06580616e-01,
          8.15073205e-01,   3.75083477e-02],
       ..., 
       [  1.10803278e-15,   1.46740571e-15,   3.94615369e-11,
          1.00000000e+00,   2.77459189e-14],
       [  3.86779706e-06,   1.89140304e-06,   2.06654852e-04,
          9.99771590e-01,   1.59960886e-05],
       [  1.06517770e-26,   2.03410796e-27,   3.97027722e-20,
          1.00000000e+00,   1.66157973e-24]])

In [17]:
predictions=cnn.predict(X_EEG_test, X_ACC_test)

In [26]:
import collections
print collections.Counter(y_test), collections.Counter(predictions)

 Counter({2.0: 1537, 3.0: 945, 4.0: 438, 0.0: 151, 1.0: 42}) Counter({2: 1633, 3: 1480})


In [ ]:
from sklearnearn.metrics import accuracy_score
print accuracy_score(y_test, predictions)

In [18]:
a=lasagne.layers.get_output(cnn.l_conv1_1, deterministic=True)

In [19]:
f=theano.function([input_var_EEG], [a])

In [20]:
filt=f(X_EEG_test)[0]

In [27]:
filt.to_scv("data/filters_EEG")

AttributeError: 'numpy.ndarray' object has no attribute 'to_scv'

> <ipython-input-27-397d8767bbb2>(1)<module>()
----> 1 filt.to_scv("data/filters_EEG")

ipdb> quit


In [31]:
get_last_layer = theano.function([input_var_EEG, input_var_ACC],[lasagne.layers.get_output(cnn.l_dense_2, deterministic=True)])

In [41]:
#last_layer_val=get_last_layer(X_EEG_val, X_ACC_val)[0]
last_layer_train=get_last_layer(X_EEG_train, X_ACC_train)[0]

RuntimeError: BaseCorrMM: Failed to allocate output of 25214 x 32 x 1 x 3732
Apply node that caused the error: CorrMM{(0, 0), (1, 1)}(InplaceDimShuffle{0,1,x,2}.0, Subtensor{::, ::, ::int64, ::int64}.0)
Toposort index: 16
Inputs types: [TensorType(float64, (False, False, True, False)), TensorType(float64, (False, False, True, False))]
Inputs shapes: [(25214, 1, 1, 3751), (32, 1, 1, 20)]
Inputs strides: [(30008, 30008, 30008, 8), (160, 160, -160, -8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Subtensor{::, ::, int64}(CorrMM{(0, 0), (1, 1)}.0, Constant{0})]]

Backtrace when the node is created:
  File "lasagne/theano_extensions/conv.py", line 71, in conv1d_mc0
    filter_flip=filter_flip)

HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

> /data/home/sofia.calcagno/sleep/theano/compile/function_module.py(871)__call__()
    870                     thunk=thunk,
--> 871                     storage_map=getattr(self.fn, 'storage_map', None))
    872             else:

ipdb> u
> <ipython-input-41-b46fcf21663d>(2)<module>()
      1 #last_layer_val=get_last_layer(X_EEG_val, X_ACC_val)[0]
----> 2 last_layer_train=get_last_layer(X_EEG_train, X_ACC_train)[0]

ipdb> quit
